# Codes

In [51]:
import numpy as np

class Simplex(object):
    def __init__(self, obj, b_matrix=[], maxloop=1000, max_mode=False):
        self.mat, self.max_mode = np.array(
            [[0] + obj]) * (-1 if max_mode else 1), max_mode
        if b_matrix != []:
            self.mat = np.vstack([self.mat, b_matrix])
        self.count = 0
        self.maxloop = maxloop

    def _pivot(self, mat, B, row, col):
        mat[row] /= mat[row][col]
        ids = np.arange(mat.shape[0]) != row
        # for each i!= row do: mat[i]= mat[i] - mat[row] * mat[i][col]
        mat[ids] -= mat[row] * mat[ids, col:col + 1]
        B[row] = col
        self.count += 1
        # print(self.count)

    def _simplex(self, mat, B, m, n):
        while mat[0, 1:].min() < 0 and self.count < self.maxloop:
            # use Bland's method to avoid degeneracy
            # col = np.where(mat[0, 1:] < 0)[0][0] + 1
            # use Dantzig's method, may encounter degeneracy
            delta = []
            for col in (np.where(mat[0, 1:] < 0)[0] + 1):
                row = np.array([mat[i][0] / mat[i][col] if mat[i][col] > 0 else 0x7fffffff for i in 
                                range(1, mat.shape[0])]).argmax() + 1
                delta.append(-mat[0][col]/mat[row][col]*mat[row][0])
                
            col = (np.where(mat[0, 1:] < 0)[0] + 1)[np.array(delta).argmin()]
            row = np.array([mat[i][0] / mat[i][col] if mat[i][col] > 0 else 0x7fffffff for i in 
                range(1, mat.shape[0])]).argmin() + 1
            if mat[row][col] <= 0:
                return None  # the theta is ∞, the problem is unbounded
            self._pivot(mat, B, row, col)
        print('loop:', self.count)
        return mat[0][0] * (1 if self.max_mode else -1), {B[i]: mat[i, 0] for i in range(1, m) if B[i] < n}

    def solve(self):
        m, n = self.mat.shape  # m - 1 is the number slack variables we should add
        temp, B = np.vstack([np.zeros((1, m - 1)), np.eye(m - 1)]
                            ), list(range(n - 1, n + m - 1))  # add diagonal array
        mat = self.mat = np.hstack([self.mat, temp])  # combine them!
        if mat[1:, 0].min() < 0:  # is the initial basic solution feasible?
            row = mat[1:, 0].argmin() + 1  # find the index of min b
            # set first row value to zero, and store the previous value
            temp, mat[0] = np.copy(mat[0]), 0
            mat = np.hstack(
                [mat, np.array([1] + [-1] * (m - 1)).reshape((-1, 1))])
            self._pivot(mat, B, row, mat.shape[1] - 1)
            if self._simplex(mat, B, m, n)[0] != 0:
                return None  # the problem has no answer
            if mat.shape[1] - 1 in B:  # if the x0 in B, we should pivot it.
                self._pivot(mat, B, B.index(
                    mat.shape[1] - 1), np.where(mat[0, 1:] != 0)[0][0] + 1)
            # recover the first line
            self.mat = np.vstack([temp, mat[1:, :-1]])
            for i, x in enumerate(B[1:]):
                self.mat[0] -= self.mat[0, x] * self.mat[i + 1]
        return self._simplex(self.mat, B, m, n)

# Test: easy

In [52]:
mt_b = [[2,1,2],[2,2,1]]
t = Simplex([-1, -2], mt_b)
print(t.solve())
print(t.mat)

loop: 2
(-2.0, {2: 0.6666666666666666, 1: 0.6666666666666667})
[[ 2.          0.          0.          1.          0.        ]
 [ 0.66666667  0.          1.          0.66666667 -0.33333333]
 [ 0.66666667  1.          0.         -0.33333333  0.66666667]]


# Test: harder

In [53]:
data = np.loadtxt("data_b.txt", dtype=int)
Z = list(data[0][1:])
b = list(data[1:])
test = Simplex(Z, b)
print(test.solve())
print(test.mat)

loop: 3
(-1350.0, {3: 30.0, 1: 400.0})
[[ 1.35e+03  0.00e+00  0.00e+00  0.00e+00  1.00e+00  2.00e+00  0.00e+00]
 [ 3.00e+01  0.00e+00 -2.00e+00  1.00e+00 -1.00e+00  1.00e+00  0.00e+00]
 [ 2.00e+01  0.00e+00  0.00e+00  0.00e+00 -2.00e+00  1.00e+00  1.00e+00]
 [ 4.00e+02  1.00e+00  4.00e+00  0.00e+00  2.00e+00 -1.00e+00  0.00e+00]]


C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


In [54]:
data = np.loadtxt("data_b.txt", dtype=int)
Z = list(data[0][1:])
b = list(data[1:])
test = Simplex(Z, b)
mat = test.mat
# np.where(mat[0, 1:] < 0)[0][0] + 1
print(mat)
np.where(mat[0, 1:] < 0)[0][0]+1

[[  0  -3  -2  -5]
 [430   1   2   1]
 [460   1   0   2]
 [420   1   4   0]]


1

# Accelerating

In [57]:
# minus_list = np.where(mat[0, 1:] < 0)[0] + 1
delta = []
for col in (np.where(mat[0, 1:] < 0)[0] + 1):
    row = np.array([mat[i][0] / mat[i][col] if mat[i][col] > 0 else 0x7fffffff for i in 
                range(1, mat.shape[0])]).argmin() + 1
    delta.append(-mat[0][col]/mat[row][col]*mat[row][0])

col = (np.where(mat[0, 1:] < 0)[0] + 1)[np.array(delta).argmax()]
row = np.array([mat[i][0] / mat[i][col] if mat[i][col] > 0 else 0x7fffffff for i in 
                range(1, mat.shape[0])]).argmin() + 1
print(col, row, delta)

1 3 [1260.0, 210.0, 1150.0]
